### Static members in interfaces

How do you abstract over operations that are inherently static – such as operators? The traditional answer is “poorly”. In C# 11 we released support for static virtual members in interfaces, which was in preview in C# 10. With this you can now define a very simple mathematical interface.

In [ ]:
public interface IMonoid<TSelf> where TSelf : IMonoid<TSelf>
{
    public static abstract TSelf operator +(TSelf a, TSelf b);
    public static abstract TSelf Zero { get; }
}

Notice how the interface takes a type parameter for “itself”. That’s because static members don’t have a this.

Anyone can now implement this interface by providing implementations for the two static members, and passing themselves as the TSelf type argument.

In [ ]:
public interface IMonoid<TSelf> where TSelf : IMonoid<TSelf>
{
    public static abstract TSelf operator +(TSelf a, TSelf b);
    public static abstract TSelf Zero { get; }
}

public struct MyInt : IMonoid<MyInt>
{
    int value;
    public MyInt(int i) => value = i;
    public static MyInt operator +(MyInt a, MyInt b) => new MyInt(a.value + b.value);
    public static MyInt Zero => new MyInt(0);
}

Importantly, how do you consume these abstract operations? How do you call virtual members when there is no instance to call them on? The answer is via generics. Here is what it looks like.

In [ ]:

public interface IMonoid<TSelf> where TSelf : IMonoid<TSelf>
{
    public static abstract TSelf operator +(TSelf a, TSelf b);
    public static abstract TSelf Zero { get; }
}

public struct MyInt : IMonoid<MyInt>
{
    int value;
    public MyInt(int i) => value = i;
    public static MyInt operator +(MyInt a, MyInt b) => new MyInt(a.value + b.value);
    public static MyInt Zero => new MyInt(0);
}

T AddAll<T>(params T[] elements) where T : IMonoid<T>
{
    T result = T.Zero;
    foreach (var element in elements)
    {
        result += element;
    }
    return result;
}

MyInt sum = AddAll<MyInt>(new MyInt(3), new MyInt(4), new MyInt(5));

In fact .NET 7 comes with a new namespace System.Numerics chock-full of math interfaces, representing the different combinations of operators and other static members that you’d ever want to use: the “grown-up” versions of the little IMonoid<T> interface above. All the numeric types in .NET now implement these new interfaces – and you can add them for your own types too! So it’s now easy to write numeric algorithms once and for all – abstracted from the concrete types they work on – instead of having forests of overloads containing essentially the same code.

Even if you do not create interfaces with static virtual members, you benefit from the improvements they make to .NET libraries, now and in the future.